<a href="https://colab.research.google.com/github/B3CODER/Time_forecasting/blob/main/Solar_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 

In [ ]:
data=pd.read_csv('SolarPrediction.csv')
print(data.head())

     UNIXTime                   Data      Time  Radiation  Temperature  \
0  1475229326  9/29/2016 12:00:00 AM  23:55:26       1.21           48   
1  1475229023  9/29/2016 12:00:00 AM  23:50:23       1.21           48   
2  1475228726  9/29/2016 12:00:00 AM  23:45:26       1.23           48   
3  1475228421  9/29/2016 12:00:00 AM  23:40:21       1.21           48   
4  1475228124  9/29/2016 12:00:00 AM  23:35:24       1.17           48   

   Pressure  Humidity  WindDirection(Degrees)  Speed TimeSunRise TimeSunSet  
0     30.46        59                  177.39   5.62    06:13:00   18:13:00  
1     30.46        58                  176.78   3.37    06:13:00   18:13:00  
2     30.46        57                  158.75   3.37    06:13:00   18:13:00  
3     30.46        60                  137.71   3.37    06:13:00   18:13:00  
4     30.46        62                  104.95   5.62    06:13:00   18:13:00  


In [ ]:
data

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,1475229326,9/29/2016 12:00:00 AM,23:55:26,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00
1,1475229023,9/29/2016 12:00:00 AM,23:50:23,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00
2,1475228726,9/29/2016 12:00:00 AM,23:45:26,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00
3,1475228421,9/29/2016 12:00:00 AM,23:40:21,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00
4,1475228124,9/29/2016 12:00:00 AM,23:35:24,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00
...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,12/1/2016 12:00:00 AM,00:20:04,1.22,44,30.43,102,145.42,6.75,06:41:00,17:42:00
32682,1480587301,12/1/2016 12:00:00 AM,00:15:01,1.17,44,30.42,102,117.78,6.75,06:41:00,17:42:00
32683,1480587001,12/1/2016 12:00:00 AM,00:10:01,1.20,44,30.42,102,145.19,9.00,06:41:00,17:42:00
32684,1480586702,12/1/2016 12:00:00 AM,00:05:02,1.23,44,30.42,101,164.19,7.87,06:41:00,17:42:00


In [ ]:
from keras.layers import Dense,Dropout,LSTM,Bidirectional 
from keras.models import Sequential

In [ ]:
sun=data[['UNIXTime','Radiation']].iloc[:31000,:]
sun.set_index('UNIXTime',drop=True,inplace=True)
sun.head()


,Radiation
UNIXTime,
1475229326,1.21
1475229023,1.21
1475228726,1.23
1475228421,1.21
1475228124,1.17


In [ ]:
training_size=round(len(sun)* 0.80)
train_data=sun[:training_size]
test_data=sun[training_size:]

In [ ]:
def create_sequence(dataset,sequence_len):
  sequences=[]
  labels=[]
  start_idx=0
  for stop_idx in range(sequence_len,len(dataset)):
    sequences.append(dataset.iloc[start_idx:stop_idx])
    labels.append(dataset.iloc[stop_idx])
    start_idx += 1
  return(np.array(sequences),np.array(labels))

In [ ]:
X_train,y_train=create_sequence(train_data,100)
X_val,y_val=create_sequence(test_data,100)

print(X_train.shape,X_val.shape)

(24700, 100, 1) (6100, 100, 1)


In [ ]:
model=Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(Dropout(0.1))
model.add(LSTM(units=50,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mean_squared_error')

In [ ]:
model.fit(X_train,y_train,epochs=25)

772/772 [==============================] - 81s 100ms/step - loss: 63152.0508


In [ ]:
from pandas import concat
from math import sqrt
from numpy import concatenate
from sklearn.metrics import mean_squared_error

yhat = model.predict(X_val)
inv_yhat = concatenate((yhat, X_val[:, 1]), axis=1)

y_val = y_val.reshape((len(y_val), 1))
inv_y = concatenate((y_val, X_val[:, 1]), axis=1)
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

191/191 [==============================] - 10s 37ms/step
Test RMSE: 51.225


WITH STANDARD SCALAR


In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from keras.layers import Dense,Dropout,LSTM,Bidirectional 
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from pandas import concat
from math import sqrt
from numpy import concatenate
from sklearn.metrics import mean_squared_error

In [ ]:
data=pd.read_csv('SolarPrediction.csv')
print(data.head())

     UNIXTime                   Data      Time  Radiation  Temperature  \
0  1475229326  9/29/2016 12:00:00 AM  23:55:26       1.21           48   
1  1475229023  9/29/2016 12:00:00 AM  23:50:23       1.21           48   
2  1475228726  9/29/2016 12:00:00 AM  23:45:26       1.23           48   
3  1475228421  9/29/2016 12:00:00 AM  23:40:21       1.21           48   
4  1475228124  9/29/2016 12:00:00 AM  23:35:24       1.17           48   

   Pressure  Humidity  WindDirection(Degrees)  Speed TimeSunRise TimeSunSet  
0     30.46        59                  177.39   5.62    06:13:00   18:13:00  
1     30.46        58                  176.78   3.37    06:13:00   18:13:00  
2     30.46        57                  158.75   3.37    06:13:00   18:13:00  
3     30.46        60                  137.71   3.37    06:13:00   18:13:00  
4     30.46        62                  104.95   5.62    06:13:00   18:13:00  


In [ ]:
sun=data[['UNIXTime','Radiation']].iloc[:31000,:]
sun.set_index('UNIXTime',drop=True,inplace=True)
sun.head()


,Radiation
UNIXTime,
1475229326,1.21
1475229023,1.21
1475228726,1.23
1475228421,1.21
1475228124,1.17


In [ ]:
MMS = MinMaxScaler()
sun[sun.columns] = MMS.fit_transform(sun)
sun.head()

,Radiation
UNIXTime,
1475229326,0.000062
1475229023,0.000062
1475228726,0.000075
1475228421,0.000062
1475228124,0.000037


In [ ]:
training_size=round(len(sun)* 0.80)
train_data=sun[:training_size]
test_data=sun[training_size:]

In [ ]:
def create_sequence(dataset,sequence_len):
  sequences=[]
  labels=[]
  start_idx=0
  for stop_idx in range(sequence_len,len(dataset)):
    sequences.append(dataset.iloc[start_idx:stop_idx])
    labels.append(dataset.iloc[stop_idx])
    start_idx += 1
  return(np.array(sequences),np.array(labels))

In [ ]:
X_train,y_train=create_sequence(train_data,100)
X_val,y_val=create_sequence(test_data,100)

print(X_train.shape,X_val.shape)

(24700, 100, 1) (6100, 100, 1)


In [ ]:
model=Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(Dropout(0.1))
model.add(LSTM(units=50,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mean_squared_error')

In [ ]:
model.fit(X_train,y_train,epochs=25)

Epoch 1/25
772/772 [==============================] - 96s 124ms/step - loss: 0.0028
Epoch 2/25
772/772 [==============================] - 83s 107ms/step - loss: 0.0027
Epoch 3/25
772/772 [==============================] - 84s 109ms/step - loss: 0.0028
Epoch 4/25
772/772 [==============================] - 82s 107ms/step - loss: 0.0028
Epoch 5/25
772/772 [==============================] - 84s 108ms/step - loss: 0.0028
Epoch 6/25
772/772 [==============================] - 82s 106ms/step - loss: 0.0028
Epoch 7/25
772/772 [==============================] - 82s 107ms/step - loss: 0.0028
Epoch 8/25
772/772 [==============================] - 83s 108ms/step - loss: 0.0027
Epoch 9/25
772/772 [==============================] - 82s 107ms/step - loss: 0.0027
Epoch 10/25
772/772 [==============================] - 82s 106ms/step - loss: 0.0027
Epoch 11/25
772/772 [==============================] - 84s 108ms/step - loss: 0.0027
Epoch 12/25
772/772 [==============================] - 82s 107ms/step - lo

In [ ]:
yhat = model.predict(X_val)
# invert scaling for forecast

inv_yhat = concatenate((yhat, X_val[:, 1]), axis=1)
inv_yhat = MMS.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
y_val = y_val.reshape((len(y_val), 1))
inv_y = concatenate((y_val, X_val[:, 1]), axis=1)
inv_y = MMS.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

191/191 [==============================] - 6s 31ms/step
Test RMSE: 61.493


Without minmaxscalar with different layer values

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from keras.layers import Dense,Dropout,LSTM,Bidirectional 
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from pandas import concat
from math import sqrt
from numpy import concatenate
from keras.layers import Dense,Dropout,LSTM,Bidirectional 
from keras.models import Sequential
from pandas import concat
from math import sqrt
from numpy import concatenate
from sklearn.metrics import mean_squared_error

data=pd.read_csv('SolarPrediction.csv')
print(data.head())

sun=data[['UNIXTime','Radiation']].iloc[:31000,:]
sun.set_index('UNIXTime',drop=True,inplace=True)
sun.head()


training_size=round(len(sun)* 0.80)
train_data=sun[:training_size]
test_data=sun[training_size:]


def create_sequence(dataset,sequence_len):
  sequences=[]
  labels=[]
  start_idx=0
  for stop_idx in range(sequence_len,len(dataset)):
    sequences.append(dataset.iloc[start_idx:stop_idx])
    labels.append(dataset.iloc[stop_idx])
    start_idx += 1
  return(np.array(sequences),np.array(labels))

X_train,y_train=create_sequence(train_data,100)
X_val,y_val=create_sequence(test_data,100)

print(X_train.shape,X_val.shape)

model=Sequential()
model.add(LSTM(units=64,return_sequences=True,input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(Dropout(0.1))
model.add(LSTM(units=32,return_sequences=False))
model.add(Dropout(0.1))
model.add(Dense(16))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mean_squared_error')

model.fit(X_train,y_train,epochs=25)

y_prediction = model.predict(X_val)
# inv_yhat = concatenate((yhat, X_val[:, 1]), axis=1)

# y_val = y_val.reshape((len(y_val), 1))
# inv_y = concatenate((y_val, X_val[:, 1]), axis=1)
# calculate RMSE
rmse = np.sqrt(((y_val - y_prediction) ** 2).mean())
print('Test RMSE: %.3f' % rmse)

     UNIXTime                   Data      Time  Radiation  Temperature  \
0  1475229326  9/29/2016 12:00:00 AM  23:55:26       1.21           48   
1  1475229023  9/29/2016 12:00:00 AM  23:50:23       1.21           48   
2  1475228726  9/29/2016 12:00:00 AM  23:45:26       1.23           48   
3  1475228421  9/29/2016 12:00:00 AM  23:40:21       1.21           48   
4  1475228124  9/29/2016 12:00:00 AM  23:35:24       1.17           48   

   Pressure  Humidity  WindDirection(Degrees)  Speed TimeSunRise TimeSunSet  
0     30.46        59                  177.39   5.62    06:13:00   18:13:00  
1     30.46        58                  176.78   3.37    06:13:00   18:13:00  
2     30.46        57                  158.75   3.37    06:13:00   18:13:00  
3     30.46        60                  137.71   3.37    06:13:00   18:13:00  
4     30.46        62                  104.95   5.62    06:13:00   18:13:00  
(24700, 100, 1) (6100, 100, 1)
Epoch 1/25
772/772 [==============================] - 13

Graph plot

In [ ]:
# plt.plot(X_val, label='True Value')
# plt.plot(y_prediction, label ='Predicted_value')
# plt.show()

Encoder-Decoder LSTM Model With Univariate Input

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from keras.layers import Dense,Dropout,LSTM,Bidirectional 
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from pandas import concat
from math import sqrt
from numpy import concatenate
from keras.layers import Dense,Dropout,LSTM,Bidirectional 
from keras.models import Sequential
from pandas import concat
from math import sqrt
from numpy import concatenate
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Conv1D, MaxPooling1D, Flatten, ConvLSTM2D, Dropout
# import tensorflow.keras.backend as K
import warnings

data=pd.read_csv('SolarPrediction.csv')
print(data.head())

sun=data[['UNIXTime','Radiation']].iloc[:31000,:]
sun.set_index('UNIXTime',drop=True,inplace=True)
sun.head()


training_size=round(len(sun)* 0.80)
train_data=sun[:training_size]
test_data=sun[training_size:]


def create_sequence(dataset,sequence_len):
  sequences=[]
  labels=[]
  start_idx=0
  for stop_idx in range(sequence_len,len(dataset)):
    sequences.append(dataset.iloc[start_idx:stop_idx])
    labels.append(dataset.iloc[stop_idx])
    start_idx += 1
  return(np.array(sequences),np.array(labels))

X_train,y_train=create_sequence(train_data,100)
X_val,y_val=create_sequence(test_data,100)

print(X_train.shape,X_val.shape)

model = Sequential()
model.add(LSTM(units=64,activation='relu',return_sequences=True,input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(LSTM(units=32,activation='relu',return_sequences=True))
model.add(LSTM(units=16,activation='relu',return_sequences=False))
model.add(RepeatVector(100))
model.add(LSTM(units=16,activation='relu',return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(loss="mse",optimizer='adam')
model.build()
print(model.summary())

model.fit(X_train,y_train,epochs=25)

y_prediction = model.predict(X_val)
rmse = np.sqrt(((y_val - y_prediction) ** 2).mean())
print('Test RMSE: %.3f' % rmse)

     UNIXTime                   Data      Time  Radiation  Temperature  \
0  1475229326  9/29/2016 12:00:00 AM  23:55:26       1.21           48   
1  1475229023  9/29/2016 12:00:00 AM  23:50:23       1.21           48   
2  1475228726  9/29/2016 12:00:00 AM  23:45:26       1.23           48   
3  1475228421  9/29/2016 12:00:00 AM  23:40:21       1.21           48   
4  1475228124  9/29/2016 12:00:00 AM  23:35:24       1.17           48   

   Pressure  Humidity  WindDirection(Degrees)  Speed TimeSunRise TimeSunSet  
0     30.46        59                  177.39   5.62    06:13:00   18:13:00  
1     30.46        58                  176.78   3.37    06:13:00   18:13:00  
2     30.46        57                  158.75   3.37    06:13:00   18:13:00  
3     30.46        60                  137.71   3.37    06:13:00   18:13:00  
4     30.46        62                  104.95   5.62    06:13:00   18:13:00  


(24700, 100, 1) (6100, 100, 1)


Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_47 (LSTM)              (None, 100, 64)           16896     
                                                                 
 lstm_48 (LSTM)              (None, 100, 32)           12416     
                                                                 
 lstm_49 (LSTM)              (None, 16)                3136      
                                                                 
 repeat_vector_17 (RepeatVec  (None, 100, 16)          0         
 tor)                                                            
                                                                 
 lstm_50 (LSTM)              (None, 100, 16)           2112      
                                                                 
 time_distributed_25 (TimeDi  (None, 100, 1)           17        
 stributed)                                          

KeyboardInterrupt: ignored

ConvLSTM Encoder-Decoder Model With Univariate Input

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from keras.layers import Dense,Dropout,LSTM,Bidirectional 
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from pandas import concat
from math import sqrt
from numpy import concatenate
from keras.layers import Dense,Dropout,LSTM,Bidirectional 
from keras.models import Sequential
from pandas import concat
from math import sqrt
from numpy import concatenate
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Conv1D, MaxPooling1D, Flatten, ConvLSTM2D, Dropout
# import tensorflow.keras.backend as K
import warnings

data=pd.read_csv('SolarPrediction.csv')
print(data.head())

sun=data[['UNIXTime','Radiation']].iloc[:31000,:]
sun.set_index('UNIXTime',drop=True,inplace=True)
sun.head()


training_size=round(len(sun)* 0.80)
train_data=sun[:training_size]
test_data=sun[training_size:]


def create_sequence(dataset,sequence_len):
  sequences=[]
  labels=[]
  start_idx=0
  for stop_idx in range(sequence_len,len(dataset)):
    sequences.append(dataset.iloc[start_idx:stop_idx])
    labels.append(dataset.iloc[stop_idx])
    start_idx += 1
  return(np.array(sequences),np.array(labels))

X_train,y_train=create_sequence(train_data,100)
X_val,y_val=create_sequence(test_data,100)

print(X_train.shape,X_val.shape)

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(RepeatVector(1))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(100, activation='relu')))
model.add(TimeDistributed(Dense(1)))
model.compile(loss='mse', optimizer='adam')
# fit network
model.fit(X_train,y_train, epochs=25)
model.summary()


y_prediction = model.predict(X_val)
rmse = np.sqrt(((y_val - y_prediction) ** 2).mean())
print('Test RMSE: %.3f' % rmse)


     UNIXTime                   Data      Time  Radiation  Temperature  \
0  1475229326  9/29/2016 12:00:00 AM  23:55:26       1.21           48   
1  1475229023  9/29/2016 12:00:00 AM  23:50:23       1.21           48   
2  1475228726  9/29/2016 12:00:00 AM  23:45:26       1.23           48   
3  1475228421  9/29/2016 12:00:00 AM  23:40:21       1.21           48   
4  1475228124  9/29/2016 12:00:00 AM  23:35:24       1.17           48   

   Pressure  Humidity  WindDirection(Degrees)  Speed TimeSunRise TimeSunSet  
0     30.46        59                  177.39   5.62    06:13:00   18:13:00  
1     30.46        58                  176.78   3.37    06:13:00   18:13:00  
2     30.46        57                  158.75   3.37    06:13:00   18:13:00  
3     30.46        60                  137.71   3.37    06:13:00   18:13:00  
4     30.46        62                  104.95   5.62    06:13:00   18:13:00  


(24700, 100, 1) (6100, 100, 1)
Epoch 1/25
772/772 [==============================] - 9s 9ms/step - loss: 10332.0664
Epoch 2/25
772/772 [==============================] - 6s 7ms/step - loss: 8352.2949
Epoch 3/25
772/772 [==============================] - 7s 9ms/step - loss: 8018.5557
Epoch 4/25
772/772 [==============================] - 5s 7ms/step - loss: 7627.9917
Epoch 5/25
772/772 [==============================] - 7s 9ms/step - loss: 7746.2671
Epoch 6/25
772/772 [==============================] - 7s 9ms/step - loss: 7335.1030
Epoch 7/25
772/772 [==============================] - 13s 17ms/step - loss: 7066.6421
Epoch 8/25
772/772 [==============================] - 12s 15ms/step - loss: 7069.6523
Epoch 9/25
772/772 [==============================] - 12s 15ms/step - loss: 6751.6265
Epoch 10/25
772/772 [==============================] - 10s 13ms/step - loss: 6512.0542
Epoch 11/25
772/772 [==============================] - 10s 13ms/step - loss: 6399.7646
Epoch 12/25
772/772 [===========